In [55]:
import pandas as pd
from herbie import Herbie
from datetime import datetime
import numpy as np
import pygrib as pg
import xarray as xr

In [56]:
date = "20190101"
date_hour = date + " 00:00"
save_dir = '/home/kaleb/data/'

pred_hours = 0

In [57]:
herb_obj = Herbie(date_hour, model="hrrr",
                  product="sfc", save_dir=save_dir,
                  verbose=True,
                  priority=['pando', 'pando2', 'aws', 'nomads'],
                  overwrite=True)
    # .xarray(":APCP:.*:(?:0-1|[1-9]\d*-\d+) hour").herbie.nearest_points(points=[(-91.0198, 30.2241)])
# herb_obj.tell_me_everything()
print(herb_obj.tell_me_everything())

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2019-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
self.DESCRIPTION=High-Resolution Rapid Refresh - CONUS
self.DETAILS={'NOMADS product description': 'https://www.nco.ncep.noaa.gov/pmb/products/hrrr/', 'University of Utah HRRR archive': 'http://hrrr.chpc.utah.edu/'}
self.EXPECT_IDX_FILE=remote
self.IDX_STYLE=wgrib2
self.LOCALFILE=hrrr.t00z.wrfsfcf00.grib2
self.PRODUCTS={'sfc': '2D surface level fields; 3-km resolution', 'prs': '3D pressure level fields; 3-km resolution', 'nat': 'Native level fields; 3-km resolution', 'subh': 'Subhourly grids; 3-km resolution'}
self.SOURCES={'pando': 'https://pando-rgw01.chpc.utah.edu/hrrr/sfc/20190101/hrrr.t00z.wrfsfcf00.grib2', 'pando2': 'https://pando-rgw02.chpc.utah.edu/hrrr/sfc/20190101/hrrr.t00z.wrfsfcf00.grib2', 'aws': 'https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20190101/conus/hrrr.t00z.wrfsfcf00.grib2'}
self.fxx=0
self.get_localFileName=hrrr.t00z.wrfsfcf00.grib2
self.get_remoteFileName=hrrr.t00z.wrfsfcf

In [58]:
herb_arr = herb_obj.xarray(":APCP:surface:")
print(herb_arr.tp.values)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [59]:
herb_obj.searchString_help

'\nUse regular expression to search for lines in the index file.\nHere are some examples you can use for the wgrib2-style `searchString`\n\n    ============================= ===============================================\n    ``searchString=``             Messages that will be downloaded\n    ============================= ===============================================\n    ":TMP:2 m"                    Temperature at 2 m.\n    ":TMP:"                       Temperature fields at all levels.\n    ":UGRD:.* mb"                 U Wind at all pressure levels.\n    ":500 mb:"                    All variables on the 500 mb level.\n    ":APCP:"                      All accumulated precipitation fields.\n    ":APCP:surface:0-[1-9]*"      Accumulated precip since initialization time\n    ":APCP:surface:[1-9]*-[1-9]*" Accumulated precip over last hour\n    ":UGRD:10 m"                  U wind component at 10 meters.\n    ":(U|V)GRD:(10|80) m"         U and V wind component at 10 and 80 m.\n    

In [60]:
close_points = herb_arr.herbie.nearest_points([(-100, 40), (-101, 41)], names=['pointa', 'pointb'])
print(close_points)

/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "tp".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


<xarray.Dataset>
Dimensions:              (point: 2)
Coordinates:
    time                 datetime64[ns] 2019-01-01
    step                 timedelta64[ns] 00:00:00
    surface              float64 0.0
    latitude             (point) float64 40.01 41.01
    longitude            (point) float64 260.0 259.0
    valid_time           datetime64[ns] 2019-01-01
    metpy_crs            object Projection: lambert_conformal_conic
    y                    (point) float64 1.707e+05 2.847e+05
    x                    (point) float64 -2.135e+05 -2.945e+05
  * point                (point) <U6 'pointa' 'pointb'
Data variables:
    tp                   (point) float32 0.0 0.0
    gribfile_projection  (point) object None None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    m

In [61]:
print(close_points.t2m.values)

AttributeError: 'Dataset' object has no attribute 't2m'

In [ ]:
print(close_points.point.values)
print(close_points.point.values[0], close_points.t2m.values[0])

In [ ]:
herb_arr = herb_obj.xarray(":(U|V)GRD:1000 mb:")
print(herb_arr)

In [ ]:
print(herb_obj.xarray(":DSWRF:surface:anl").nearest_points([-100, 40]).values)
# print(rad)


In [62]:
# df = rad.to_dataframe()

In [63]:
dt = "20190101 10:00"
dtobj = datetime.strptime(dt, "%Y%m%d %H:%M")
herb_obj = Herbie(dtobj, model="hrrr",
                  product="sfc", save_dir=save_dir,
                  verbose=True,
                  priority=['pando', 'pando2', 'aws', 'nomads'],
                  overwrite=True)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2019-Jan-01 10:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [64]:
# print(herb_obj.xarray(":DSWRF:surface:anl").herbie.nearest_points([(-91.46, 32.145)]).values)
arr = herb_obj.xarray(":DSWRF:surface:anl").herbie.nearest_points([(-91.46, 32.145)])

/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "dswrf".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


In [65]:
print(arr.dswrf.values)

[0.]


In [66]:
name_arr = ['p1', 'p2']
values_arr = [(-100, 42), (-101, 41)]
arr = herb_obj.xarray(":DSWRF:surface:anl").herbie.nearest_points(points=values_arr, names=name_arr)
print(arr)

<xarray.Dataset>
Dimensions:              (point: 2)
Coordinates:
    time                 datetime64[ns] 2019-01-01T10:00:00
    step                 timedelta64[ns] 00:00:00
    surface              float64 0.0
    latitude             (point) float64 42.0 41.01
    longitude            (point) float64 260.0 259.0
    valid_time           datetime64[ns] 2019-01-01T10:00:00
    metpy_crs            object Projection: lambert_conformal_conic
    y                    (point) float64 3.927e+05 2.847e+05
    x                    (point) float64 -2.075e+05 -2.945e+05
  * point                (point) <U2 'p1' 'p2'
Data variables:
    dswrf                (point) float32 0.0 0.0
    gribfile_projection  (point) object None None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - N

/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "dswrf".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


In [68]:
gust_arr = herb_obj.xarray(":GUST:").herbie.nearest_points(points=values_arr, names=name_arr)
print(gust_arr)

<xarray.Dataset>
Dimensions:              (point: 2)
Coordinates:
    time                 datetime64[ns] 2019-01-01T10:00:00
    step                 timedelta64[ns] 00:00:00
    surface              float64 0.0
    latitude             (point) float64 42.0 41.01
    longitude            (point) float64 260.0 259.0
    valid_time           datetime64[ns] 2019-01-01T10:00:00
    metpy_crs            object Projection: lambert_conformal_conic
    y                    (point) float64 3.927e+05 2.847e+05
    x                    (point) float64 -2.075e+05 -2.945e+05
  * point                (point) <U2 'p1' 'p2'
Data variables:
    gust                 (point) float32 12.05 12.42
    gribfile_projection  (point) object None None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service

/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gust".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


In [69]:
gust_arr.gust.values

array([12.0498495, 12.4248495], dtype=float32)

In [20]:
dict = {'apples' : {'boy' : ['girl', 'wumbo']}, 'oranges' : {'he' : 'she'}}
print(dict)

{'apples': {'boy': ['girl', 'wumbo']}, 'oranges': {'he': 'she'}}


In [21]:
print(dict['apples'])

{'boy': ['girl', 'wumbo']}


In [22]:
print(dict['apples']['boy'])

['girl', 'wumbo']


In [23]:
dict['apples']['boy'].append(['other', 'muhlady'])
print(dict['apples']['boy'])

['girl', 'wumbo', ['other', 'muhlady']]


In [25]:
print(dict['apples']['boy'][2][0])


other


In [45]:
file_path = "../myFiles/countyInfo/us-state-ansi-fips2.csv"
df = pd.read_csv(file_path)

In [54]:
print(df.index[df['st']==1].tolist()[0])

0


In [43]:
def func(val):
    val = str(val)
    val += 'edittttt'
    return val
df['st'] = df['st'].apply(func)
print(df['st'])

0      1edittttt
1      2edittttt
2      4edittttt
3      5edittttt
4      6edittttt
5      8edittttt
6      9edittttt
7     10edittttt
8     11edittttt
9     12edittttt
10    13edittttt
11    15edittttt
12    16edittttt
13    17edittttt
14    18edittttt
15    19edittttt
16    20edittttt
17    21edittttt
18    22edittttt
19    23edittttt
20    24edittttt
21    25edittttt
22    26edittttt
23    27edittttt
24    28edittttt
25    29edittttt
26    30edittttt
27    31edittttt
28    32edittttt
29    33edittttt
30    34edittttt
31    35edittttt
32    36edittttt
33    37edittttt
34    38edittttt
35    39edittttt
36    40edittttt
37    41edittttt
38    42edittttt
39    44edittttt
40    45edittttt
41    46edittttt
42    47edittttt
43    48edittttt
44    49edittttt
45    50edittttt
46    51edittttt
47    53edittttt
48    54edittttt
49    55edittttt
50    56edittttt
Name: st, dtype: object


In [70]:
f = open("testingggg.txt", 'wt')

In [71]:
f.write("ball")

4

In [72]:
f.write(', other ball')

12

In [73]:
f.close()

In [74]:
f = open("testingggg.txt", 'a')
f.write(", other other ball")
f.close()

In [75]:
f = open("testingggg.txt", 'a')
f.write("\n other other other ball")
f.close()

In [18]:
from datetime import date, timedelta, datetime
d1 = datetime.datetime.strptime("20190101 10:00", "%Y%m%d %H:%M")
d2 = datetime.strptime("20190101 12:00", "%Y%m%d %H:%M")

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [19]:
print(d1.strftime("%m"))

01


In [35]:
begin_hour = datetime.strptime("00:00", "%H:%M")
end_hour = datetime.strptime("23:00", "%H:%M")
begin_date = datetime.strptime("20230202", "%Y%m%d")
end_date = datetime.strptime("20230204", "%Y%m%d")

In [36]:
hourrange = (end_hour - begin_hour).seconds // (60*60)
daterange = (end_date - begin_date).days
print(daterange, hourrange)

2 23


In [38]:
for i in range(0, (daterange)):
    for j in range(0, (hourrange)):
        currday = begin_date + timedelta(days=i, hours=j)
        print(currday)

2023-02-02 00:00:00
2023-02-02 01:00:00
2023-02-02 02:00:00
2023-02-02 03:00:00
2023-02-02 04:00:00
2023-02-02 05:00:00
2023-02-02 06:00:00
2023-02-02 07:00:00
2023-02-02 08:00:00
2023-02-02 09:00:00
2023-02-02 10:00:00
2023-02-02 11:00:00
2023-02-02 12:00:00
2023-02-02 13:00:00
2023-02-02 14:00:00
2023-02-02 15:00:00
2023-02-02 16:00:00
2023-02-02 17:00:00
2023-02-02 18:00:00
2023-02-02 19:00:00
2023-02-02 20:00:00
2023-02-02 21:00:00
2023-02-02 22:00:00
2023-02-03 00:00:00
2023-02-03 01:00:00
2023-02-03 02:00:00
2023-02-03 03:00:00
2023-02-03 04:00:00
2023-02-03 05:00:00
2023-02-03 06:00:00
2023-02-03 07:00:00
2023-02-03 08:00:00
2023-02-03 09:00:00
2023-02-03 10:00:00
2023-02-03 11:00:00
2023-02-03 12:00:00
2023-02-03 13:00:00
2023-02-03 14:00:00
2023-02-03 15:00:00
2023-02-03 16:00:00
2023-02-03 17:00:00
2023-02-03 18:00:00
2023-02-03 19:00:00
2023-02-03 20:00:00
2023-02-03 21:00:00
2023-02-03 22:00:00


In [53]:
import pandas as pd
df = pd.read_csv('/home/kaleb/Desktop/full_preprocessing_output/daily_data/2023/LA/HRRR_22_LA_2023-01.csv', index_col=False, na_filter=False, na_values='N/A')

print(df)


      Year  Month  Day Daily/Monthly State      County  FIPS Code  Grid Index  \
0     2023      1   31         Daily    LA  LAFAYETTE       22055           0   
1     2023      1   31         Daily    LA  LAFAYETTE       22055           1   
2     2023      1   31         Daily    LA  LAFAYETTE       22055           2   
3     2023      1   31         Daily    LA  LAFAYETTE       22055           3   
4     2023      1   31         Daily    LA  LAFAYETTE       22055           4   
...    ...    ...  ...           ...   ...         ...        ...         ...   
2134  2023      1   31         Daily    LA     TENSAS       22107          17   
2135  2023      1   31         Daily    LA     TENSAS       22107          18   
2136  2023      1   31         Daily    LA     TENSAS       22107          19   
2137  2023      1   31         Daily    LA     TENSAS       22107          20   
2138  2023      1   31         Daily    LA     TENSAS       22107          21   

      Lat (llcrnr)  Lon (ll

In [4]:
import numpy as np
names = [1,2,3,4,45]
arr = np.zeros((len(names),))

print(arr)

[0. 0. 0. 0. 0.]


In [20]:
from datetime import datetime
from herbie import Herbie
dtobj = datetime.strptime("20190101 00:00", "%Y%m%d %H:%M")
herb = Herbie(dtobj, model='hrrr',
                              product='sfc', save_dir='/home/kaleb/', verbose=True,
                              priority=['pando', 'pando2', 'aws', 'nomads',
                                        'google', 'azure', 'ecmwf', 'aws-old'],
                              fxx=0,
                              overwrite=False)


ERROR 1: PROJ: proj_create_from_database: Open of /home/kaleb/miniconda3/envs/postproc/share/proj failed


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2019-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [39]:
lon_lats = [(-99, 32), (-100, 33)]
grid_names = ['a', 'b']
dict = {'xx': 33}
xx = herb.xarray(":(?:TMP|RH):2 m").assign_coords(coords=dict).xx
print(xx)

<xarray.DataArray 'xx' ()>
array(33)
Coordinates:
    time               datetime64[ns] 2019-01-01
    step               timedelta64[ns] 00:00:00
    heightAboveGround  float64 2.0
    valid_time         datetime64[ns] 2019-01-01
    xx                 int64 33


In [38]:
temp_rh_obj = herb.xarray(":(?:TMP|RH):2 m").herbie.nearest_points(points=lon_lats, names=grid_names)
print(temp_rh_obj)

/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "t2m".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "r2".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'
/home/kaleb/miniconda3/envs/postproc/lib/python3.9/site-packages/metpy/xarray.py:382: UserWarning: More than one time coordinate present for variable "gribfile_projection".
  warnings.warn('More than one ' + axis + ' coordinate present for variable'


<xarray.Dataset>
Dimensions:              (point: 2)
Coordinates:
    time                 datetime64[ns] 2019-01-01
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 2.0
    latitude             (point) float64 32.0 33.0
    longitude            (point) float64 261.0 260.0
    valid_time           datetime64[ns] 2019-01-01
    metpy_crs            object Projection: lambert_conformal_conic
    y                    (point) float64 -7.233e+05 -6.093e+05
    x                    (point) float64 -1.415e+05 -2.345e+05
  * point                (point) <U1 'a' 'b'
Data variables:
    t2m                  (point) float32 285.3 285.6
    r2                   (point) float32 36.5 32.7
    gribfile_projection  (point) object None None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:        

In [8]:
'''
Find all fips codes of counties of a given state, given the abbreviation
'''
args = "state=LA,MS,IA,IL"
sep = args.split('=')
states = sep[1].split(',')
print(states)

['LA', 'MS', 'IA', 'IL']


In [9]:
for i in range(len(states)):
    states[i] = states[i].strip()
print(states)

['LA', 'MS', 'IA', 'IL']


In [10]:
import pandas as pd
all_county_df = pd.read_csv("../myFiles/countyInfo/countyFipsandCoordinates.csv", dtype=str)
state_abbrev_df = pd.read_csv("../myFiles/countyInfo/us-state-ansi-fips2.csv", dtype=str)
# print(all_county_csv.head())

In [12]:
state_fips = []
county_fips = []
for state in states:
    this_state_fips = state_abbrev_df['st'].where(state_abbrev_df['stusps'] == state).dropna().values[0]
    county_fips_from_state = all_county_df[all_county_df['fips_code'].str.startswith(this_state_fips)]
    county_fips_from_state = county_fips_from_state['fips_code'].dropna().values
    # county_fips_from_state = county_fips_from_state[0:len(county_fips_from_state)-2]
    state_fips.append(this_state_fips)
    county_fips.append(county_fips_from_state)
print(state_fips)
# print(county_fips)
# for val in county_fips:
#     print(val)
string_out = '--fips '
for arr in county_fips:
    for val in arr:
        string_out += val + ' '
print(string_out)

['22', '28', '19', '17']
--fips 22007 22017 22027 22061 22055 22005 22089 22113 22123 22109 22011 22041 22107 22093 22079 22099 22013 22125 22015 22081 22003 22083 22127 22117 22053 22111 22069 22087 22101 22095 22009 22065 22115 22031 22035 22021 22037 22045 22047 22119 22067 22071 22073 22063 22001 22025 22097 22105 22019 22049 22051 22033 22077 22029 22103 22091 22057 22023 22121 22075 22059 22039 22043 22085 28099 28117 28107 28131 28003 28039 28129 28095 28035 28081 28031 28069 28015 28077 28123 28005 28023 28057 28103 28075 28115 28161 28159 28037 28085 28097 28101 28067 28017 28111 28079 28061 28009 28137 28105 28113 28071 28001 28139 28049 28065 28053 28145 28019 28029 28055 28133 28087 28073 28059 28025 28135 28147 28143 28157 28013 28021 28041 28047 28127 28027 28151 28043 28091 28125 28153 28063 28089 28119 28011 28141 28093 28109 28149 28051 28007 28045 28033 28121 28083 28163 28155 19109 19031 19069 19027 19075 19165 19197 19025 19187 19087 19175 19067 19137 19147 19047 19

In [16]:
arrr = string_out.split(' ')
print((arr))

['17189' '17005' '17107' '17095' '17079' '17115' '17117' '17123' '17029'
 '17037' '17041' '17049' '17053' '17047' '17137' '17109' '17121' '17175'
 '17199' '17141' '17101' '17135' '17059' '17055' '17093' '17039' '17011'
 '17197' '17171' '17187' '17051' '17153' '17035' '17023' '17163' '17147'
 '17097' '17169' '17045' '17009' '17043' '17017' '17063' '17201' '17007'
 '17001' '17103' '17173' '17111' '17061' '17157' '17019' '17159' '17131'
 '17191' '17027' '17075' '17065' '17067' '17099' '17091' '17139' '17185'
 '17113' '17143' '17151' '17069' '17015' '17057' '17183' '17033' '17089'
 '17125' '17025' '17021' '17073' '17129' '17133' '17149' '17083' '17119'
 '17081' '17155' '17105' '17167' '17203' '17087' '17165' '17013' '17195'
 '17181' '17003' '17071' '17031' '17177' '17161' '17179' '17145' '17127'
 '17077' '17085' '17193']


In [18]:
herb = None
if herb:
    print('herb')

In [1]:
import pandas as pd
import numpy as np
dict = {'col1' : [0,1,2,3,4,5,6,7,8,8,8,8,0,0,0,4,4,4],
       'col2' : [3,4,3,4,5,3,5,5,3,5,6,7,0,0,4,3,2,0]}
df = pd.DataFrame(dict)

In [2]:
col1 = df['col1']
print(col1)
col1.replace(0, np.nan, inplace=True)
col1.ffill(inplace=True)
col1.bfill(inplace=True)
col1 = col1.to_numpy().astype(dtype=float)
print(col1)

0     0
1     1
2     2
3     3
4     4
5     5
6     6
7     7
8     8
9     8
10    8
11    8
12    0
13    0
14    0
15    4
16    4
17    4
Name: col1, dtype: int64
[1. 1. 2. 3. 4. 5. 6. 7. 8. 8. 8. 8. 8. 8. 8. 4. 4. 4.]


In [4]:
arr = np.full((4,), -12345.678)
print(arr)

[-12345.678 -12345.678 -12345.678 -12345.678]


In [20]:
import numpy as np
a = np.empty((1,))
print(a)
a = np.delete(a, 0)
print(a)
b = np.array([8, 2, 4])

c = np.concatenate((a,b))
print(c)

[4.]
[]
[8. 2. 4.]


In [ ]:
for i in range(0,6,2):
    print(i)